In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'

In [ ]:
import addict
import datetime
import os
import pyproj
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from okada_wrapper import dc3dwrapper

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [ ]:
import celeri
celeri = reload(celeri)

In [ ]:
RUN_NAME = datetime.datetime.now().strftime("%y%m%d%H%M%S") + os.sep
command_file_name = './data/western_north_america/western_north_america_command.json'
# command_file_name = './data/global/global_command.json'
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [ ]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)

In [ ]:
def fault_parameters_to_okada_format(sx1, sy1, sx2, sy2, dip, D, bd):
    """
    This function takes fault trace, dip, and locking depth information
    and calculates the anchor coordinates, length, width and strike of
    the fault plane following (1985).

    Arguments:
        sx1 : x coord of fault trace endpoint 1
        sy1 : y coord of fault trace endpoint 1
        sx2 : x coord of fault trace endpoint 2
        sy2 : y coord of fault trace endpoint 2
        dip : dip of fault plane (degrees)
        D : fault locking depth
        bd : burial depth (top "locking depth")

    Returned variables:
        strike : stike of fault plane
        L : fault length
        W : fault width
        ofx : x coord of fault anchor
        ofy : y coord of fault anchor
        ofxe : x coord of other buried corner
        ofye : y coord of other buried corner
        tfx : x coord of fault anchor (top relative)
        tfy : y coord of fault anchor (top relative)
        tfxe : x coord of other buried corner (top relative)
        tfye : y coord of other buried corner (top relative)
    """

    okada_parameters = addict.Dict()
    okada_parameters.strike = np.arctan2(sy1 - sy2, sx1 - sx2) + np.pi # This is by convention
    okada_parameters.L = np.sqrt((sx2 - sx1)**2 + (sy2 - sy1)**2)
    okada_parameters.W = (D - bd) / np.sin(np.deg2rad(dip))

    # Calculate fault segment anchor and other buried point
    okada_parameters.ofx = sx1 + D / np.tan(np.deg2rad(dip)) * np.sin(np.deg2rad(okada_parameters.strike))
    okada_parameters.ofy = sy1 - D / np.tan(np.deg2rad(dip)) * np.cos(np.deg2rad(okada_parameters.strike))
    okada_parameters.ofxe = sx2 + D / np.tan(np.deg2rad(dip)) * np.sin(np.deg2rad(okada_parameters.strike))
    okada_parameters.ofye = sy2 - D / np.tan(np.deg2rad(dip)) * np.cos(np.deg2rad(okada_parameters.strike))

    # Calculate fault segment anchor and other buried point (top relative)
    okada_parameters.tfx = sx1 + bd / np.tan(np.deg2rad(dip)) * np.sin(np.deg2rad(okada_parameters.strike))
    okada_parameters.tfy = sy1 - bd / np.tan(np.deg2rad(dip)) * np.cos(np.deg2rad(okada_parameters.strike))
    okada_parameters.tfxe = sx2 + bd / np.tan(np.deg2rad(dip)) * np.sin(np.deg2rad(okada_parameters.strike))
    okada_parameters.tfye = sy2 - bd / np.tan(np.deg2rad(dip)) * np.cos(np.deg2rad(okada_parameters.strike))

    return okada_parameters


# def GetElasticPartials(segment, stations):
"""
Calculates the elastic displacement partial derivatives based on the Okada
formulation, using the source and receiver geometries defined in
dicitonaries segment and stations. Before calculating the partials for
each segment, a local oblique Mercator project is done.
"""

n_stations = len(station)
n_segments = len(segment)
G = np.zeros((3 * n_stations, 3 * n_segments))
v1 = np.zeros(n_segments)
v2 = np.zeros(n_segments)
v3 = np.zeros(n_segments)

# Loop through each segment and calculate displacements
# for i in range(n_segments):
for i in range(1):
    print(i)

    # Local map projection
    projection = celeri.get_segment_oblique_projection(segment.lon1[i], segment.lat1[i], segment.lon2[i], segment.lat2[i])
    station_x, station_y = projection(station.lon, station.lat)
    segment_x1, segment_y1 = projection(segment.lon1[i], segment.lat1[i])
    segment_x2, segment_y2 = projection(segment.lon2[i], segment.lat2[i])

    # Calculate fault parameters in Okada form
    # okada_parameters = fault_parameters_to_okada_format(f.px1, f.py1, f.px2, f.py2, segment.dip[i], segment.locking_depth[i], segment.burial_depth[i])
    okada_parameters = fault_parameters_to_okada_format(segment_x1, segment_y1,
        segment_x2, segment_y2, segment.dip[i], segment.locking_depth[i], 
        segment.burial_depth[i])

    # Translate observation coordinates relative to fault anchor

    # Rotation observation coordinates to remove strike.
    # rot = np.array([[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
    rotation_matrix = np.array([[np.cos(np.deg2rad(okada_parameters.strike)), -np.sin(np.deg2rad(okada_parameters.strike))], 
                                [np.sin(np.deg2rad(okada_parameters.strike)), np.cos(np.deg2rad(okada_parameters.strike))]])

#     % Displacements due to unit slip components
#     [ves vns vus...
#      ved vnd vud...
#      vet vnt vut] = okada_partials(ofx, ofy, strike, f.lDep, deg_to_rad(f.dip), L, W, 1, 1, 1, s.fpx, s.fpy, command.poissons_ratio);
     
#     v1{i} = reshape([ves vns vus]', 3*nSta, 1)
#     v2{i} = reshape(sign(90 - f.dip).*[ved vnd vud]', 3*nSta, 1)
#     v3{i} = reshape((f.dip - 90 == 0).*[vet vnt vut]', 3*nSta, 1) 
#     v1{i} = xyz2enumat((v1{i}), -f.strike + 90)
#     v2{i} = xyz2enumat((v2{i}), -f.strike + 90)
#     v3{i} = xyz2enumat((v3{i}), -f.strike + 90)

# Place cell arrays into the partials matrix
# G[:, 0::3] = cell2mat(v1)
# G[:, 1::3] = cell2mat(v2)
# G[:, 2::3] = cell2mat(v3) 


In [183]:
# TODO: Locking depths are currently meters rather than KM in inputfiles!!!

i = 0
segment.locking_depth.values[i] *= celeri.KM2M
segment.burial_depth.values[i] *= celeri.KM2M

# Project coordinates to flat space using a local oblique Mercator projection
projection = celeri.get_segment_oblique_projection(segment.lon1[i], segment.lat1[i], segment.lon2[i], segment.lat2[i], True)
station_x, station_y = projection(station.lon, station.lat)
segment_x1, segment_y1 = projection(segment.lon1[i], segment.lat1[i])
segment_x2, segment_y2 = projection(segment.lon2[i], segment.lat2[i])

# Calculate geometric fault parameters
segment_strike = np.arctan2(segment_y2 - segment_y1, segment_x2 - segment_x1) # radians
segment_length = np.sqrt((segment_y2 - segment_y1)**2.0 + (segment_x2 - segment_x1)**2.0)
segment_up_dip_width = (segment.locking_depth[i] - segment.burial_depth[i]) / np.sin(np.deg2rad(segment.dip[i]))

# Translate stations and segment so that segment mid-point is at the origin
segment_x_mid = (segment_x1 + segment_x2) / 2.0
segment_y_mid = (segment_y1 + segment_y2) / 2.0
station_x -= segment_x_mid
station_y -= segment_y_mid
segment_x1 -= segment_x_mid
segment_x2 -= segment_x_mid
segment_y1 -= segment_y_mid
segment_y2 -= segment_y_mid

# Unrotate coordinates to eliminate strike, segment will lie along y = 0
# np.einsum guidance from: https://stackoverflow.com/questions/26289972/use-numpy-to-multiply-a-matrix-across-an-array-of-points
rotation_matrix = np.array([[np.cos(segment_strike), -np.sin(segment_strike)], 
                            [np.sin(segment_strike), np.cos(segment_strike)]])
un_rotation_matrix = np.array([[np.cos(-segment_strike), -np.sin(-segment_strike)], 
                               [np.sin(-segment_strike), np.cos(-segment_strike)]])
station_x_rotated, station_y_rotated = np.hsplit(np.einsum("ij,kj->ik", np.dstack((station_x, station_y))[0], un_rotation_matrix), 2)
segment_x1_rotated, segment_y1_rotated = un_rotation_matrix.dot([segment_x1, segment_y1])
segment_x2_rotated, segment_y2_rotated = un_rotation_matrix.dot([segment_x2, segment_y2])

# Elastic displacements due to fault slip from Okada 1985
alpha = (command.material_lambda + command.material_mu) / (command.material_lambda + 2 * command.material_mu)
u_x = np.zeros_like(station_x)
u_y = np.zeros_like(station_x)
u_z = np.zeros_like(station_x)
for j in range(len(station)):
    _, u, _ = dc3dwrapper(alpha, # (lambda + mu) / (lambda + 2 * mu)
                          [station_x_rotated[j], station_y_rotated[j], 0], # (meters) observation point
                          -segment.locking_depth[i], # (meters) depth of the fault origin
                          segment.dip[i], # (degrees) the dip-angle of the rectangular dislocation surface
                          [-segment_length / 2, segment_length / 2], # (meters) the along-strike range of the surface (al1,al2 in the original)
                          [0, segment_up_dip_width], # (meters) along-dip range of the surface (aw1, aw2 in the original)
                          [1.0, 0.0, 0.0]) # (meters) strike-slip, dip-slip, tensile-slip
    u_x[j] = u[0]
    u_y[j] = u[1]
    u_z[j] = u[2]



# plt.figure()
# plt.plot(station.lon, station.lat, '.b', markersize=1)
# plt.plot([segment.lon1[i], segment.lon2[i]], [segment.lat1[i], segment.lat2[i]], "-r")
# plt.show()

plt.figure()
plt.plot([segment_x1, segment_x2], [segment_y1, segment_y2], "-r")
plt.plot(station_x, station_y, '.b', markersize=1)
plt.xlim([-1e6, 1e6])
plt.ylim([-1e6, 1e6])
plt.gca().set_aspect('equal', adjustable='box')
plt.title("Positions: projected and translated")
plt.show()

plt.figure()
plt.plot([segment_x1_rotated, segment_x2_rotated], [segment_y1_rotated, segment_y2_rotated], "-r")
plt.plot(station_x_rotated, station_y_rotated, '.b', markersize=1)
plt.xlim([-1e6, 1e6])
plt.ylim([-1e6, 1e6])
plt.gca().set_aspect('equal', adjustable='box')
plt.title("Positions: projected, translated, and rotated")
plt.show()

plt.figure()
plt.plot([segment_x1_rotated, segment_x2_rotated], [segment_y1_rotated, segment_y2_rotated], "-r")
# plt.plot(station_x_rotated, station_y_rotated, '.b', markersize=1)
plt.quiver(station_x_rotated, station_y_rotated, u_x, u_y, scale=1e-1, scale_units='inches')
plt.xlim([-1e6, 1e6])
plt.ylim([-1e6, 1e6])
plt.gca().set_aspect('equal', adjustable='box')
plt.title("Displacements: projected, translated, and rotated")
plt.show()





Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [186]:
plt.figure()
plt.plot(u_x)
plt.show()
print(u_x)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[nan nan nan ... nan nan nan]


In [ ]:
c[i]
# Zoom on region of interest

# Plot corners and achor point

# Plot strike-slip deformation

# Plot dip-slip deformation

# Plot tensile-slip_deformation

In [ ]:
# Try Ben's test Okada
"""
Seven arguments are required:

alpha = (lambda + mu) / (lambda + 2 * mu)
xo = 3-vector representing the observation point (x, y, z in the original)
depth = the depth of the fault origin
dip = the dip-angle of the rectangular dislocation surface
strike_width = the along-strike range of the surface (al1,al2 in the original)
dip_width = the along-dip range of the surface (aw1, aw2 in the original)
dislocation = 3-vector representing the direction of motion on the surface
(DISL1 = strike-slip, DISL2 = dip-slip, DISL3 = opening/overlap)
"""

source_depth = 3.0e3 # meters
dip = 90  # degrees
alpha = (command.material_lambda + command.material_mu) / (command.material_lambda + 2 * command.material_mu)

n_obs = 100
x = np.linspace(-1, 1, n_obs)
y = np.linspace(-1, 1, n_obs)
x, y = np.meshgrid(x, y)
x = x.flatten()
y = y.flatten()

# x = x - x_fault
# y = y - yfault

ux = np.zeros_like(x)
uy = np.zeros_like(x)
uz = np.zeros_like(x)
for i in range(x.size):
    _, u, _ = dc3dwrapper(alpha, # (lambda + mu) / (lambda + 2 * mu)
                            [x[i], y[i], 0], # (meters) observation point
                            source_depth, # (meters) depth of the fault origin
                            dip, # (degrees) the dip-angle of the rectangular dislocation surface
                            [-0.6, 0.6], # (meters) the along-strike range of the surface (al1,al2 in the original)
                            [0, 3.0], # (meters) along-dip range of the surface (aw1, aw2 in the original)
                            [1.0, 0.0, 0.0]) # (meters) strike-slip, dip-slip, tensile-slip
    ux[i] = u[0]
    uy[i] = u[1]
    uz[i] = u[2]


plt.figure()
plt.quiver(x, y, ux, uy)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()



In [ ]:
# def okada_partials(xf, yf, strike, d, delta, L, W, U1, U2, U3, xs, ys, Pr):
#   """
#   Arguments:
#     xf     : x component of fault corner in Okada ref frame
#     xf     : y component of fault corner in Okada ref frame
#     strike : is the azimuth of the fault (should always be zero for blocks_sp1 case)
#     d      : is the depth (-z) of the origin of the fault
#     dip    : is the inclination of the fault (measured clockwise from horizontal left, facing along the strike)
#     L      : is the along strike length of the fault plane
#     W      : is the down dip length of fault plane
#     U1     : is the magnitude of the strike slip dislocation
#     U2     : is the magnitude of the dip slip dislocation
#     U3     : is the magnitude of the tensile dislocation
#     xs     : x component of station position
#     ys     : y component of station position
#     Pr     : is Poisson's ratio

#   Returned variables:
#     uxtot  : total x displacement
#     uytot  : total y displacement
#     uztot  : total z displacement

#   """
#   uxtot			       = zeros(length(xs), 1);
#   uytot              = zeros(length(xr), 1);
#   uztot              = zeros(length(xr), 1);

#   tol                = 1.0e-4
#   alpha              = -2 * Pr + 1

#   # Get station locations relative to fault anchor and remove strike
#   xt                 = xs - xf
#   yt                 = ys - yf

#   alpha_rot          = -strike
#   xr, yr           = rotate_xy_vec(xt, yt, alpha_rot)


# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %%  Calculate some values that are frequently needed  %%
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# sind               = sin(delta);
# cosd               = cos(delta);
# twopi              = 2.*pi;


# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
# %%  Find displacements at each station  %%
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#   x               = xr;
#   y               = yr;
#   p               = y.*cosd + d.*sind;
#   q               = repmat(y.*sind - d.*cosd, 1, 4);
#   zi              = [x x x-L x-L];
#   eta             = [p p-W p p-W];
#   ybar            = eta.*cosd + q.*sind;
#   dbar            = eta.*sind - q.*cosd;
#   R               = sqrt(zi.^2 + eta.^2 + q.^2);
#   X               = sqrt(zi.^2 + q.^2);

#   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#   %%  Calculate some more commonly used values  %%
#   %%  These are introduced to reduce repetive   %%
#   %%  calculations.  (see okada.m for Okada's   %%
#   %%  form of the equations)                    %%
#   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#   Reta            = R+eta;
#   Rzi             = R+zi;
#   Rdbar           = R+dbar;
#   qdivR           = q./R;
#   phi             = atan(zi.*eta./q./R);


#   if (abs(cosd) >= tol)
#     I5            =  alpha * 2 ./ cosd * atan((eta.*(X + q.*cosd) ...
#                      + (X.*(R + X)*sind))./(zi.*(R+X).*cosd));
#     I4            =  alpha./cosd * (log(Rdbar) - sind.*log(Reta));
#     I3            =  alpha .* (1./cosd.*ybar./Rdbar - log(Reta) )  ...
#                      + sind./cosd.*I4;
#     I2            =  alpha .* (-log(Reta)) - I3;
#     I1            =  alpha .* (-1./cosd.*zi./Rdbar) - sind./cosd.*I5;
#   else
#     I5            = -alpha.*(zi.*sind)./Rdbar;
#     I4            = -alpha.*q./Rdbar;
#     I3            =  alpha./2 .*(eta./Rdbar + ybar.*q./Rdbar.^2 - log(Reta));
#     I2            =  alpha .* (-log(Reta)) - I3;
#     I1            = -alpha/2 .* (zi.*q)./Rdbar.^2;
#   end

#   uxs             = -U1./twopi .* (zi.*qdivR./(Reta) + phi + I1.*sind);
#   uxd             = -U2./twopi .* (qdivR - I3.*sind.*cosd);
#   uxt             =  U3./twopi .* (q.*qdivR./(Reta) - I3.*sind.^2);

#   uys             = -U1./twopi .* (ybar.*qdivR./(Reta) + q.*cosd./(Reta) + I2.*sind);
#   uyd             = -U2./twopi .* (ybar.*qdivR./(Rzi)  + cosd.*phi - I1.*sind.*cosd);
#   uyt             =  U3./twopi .* (-dbar.*qdivR./(Rzi) - sind.*(zi.*qdivR./(Reta) - phi) - I1.*sind.^2);

#   uzs             = -U1./twopi .* (dbar.*qdivR./(Reta) + q.*sind./(Reta) + I4.*sind);
#   uzd             = -U2./twopi .* (dbar.*qdivR./(Rzi) + sind.*phi - I5.*sind.*cosd);
#   uzt             =  U3./twopi .* (ybar.*qdivR./(Rzi) + cosd.*(zi.*qdivR./(Reta) - phi) - I5.*sind.^2);

#   uxstot          = uxs(:, 1) - uxs(:, 2) - uxs(:, 3) + uxs(:, 4);
#   uxdtot          = uxd(:, 1) - uxd(:, 2) - uxd(:, 3) + uxd(:, 4);
#   uxttot          = uxt(:, 1) - uxt(:, 2) - uxt(:, 3) + uxt(:, 4);
#   uystot          = uys(:, 1) - uys(:, 2) - uys(:, 3) + uys(:, 4);
#   uydtot          = uyd(:, 1) - uyd(:, 2) - uyd(:, 3) + uyd(:, 4);
#   uyttot          = uyt(:, 1) - uyt(:, 2) - uyt(:, 3) + uyt(:, 4);
#   uzstot          = uzs(:, 1) - uzs(:, 2) - uzs(:, 3) + uzs(:, 4);
#   uzdtot          = uzd(:, 1) - uzd(:, 2) - uzd(:, 3) + uzd(:, 4);
#   uzttot          = uzt(:, 1) - uzt(:, 2) - uzt(:, 3) + uzt(:, 4);

# %%  Rotate the station displacements back to include the effect of the strike
# [uxstot, uystot]  = rotate_xy_vec(uxstot, uystot, -alpha_rot);
# [uxdtot, uydtot]  = rotate_xy_vec(uxdtot, uydtot, -alpha_rot);
# [uxttot, uyttot]  = rotate_xy_vec(uxttot, uyttot, -alpha_rot);

# return uxstot, uystot, uzstot, uxdtot, uydtot, uzdtot, uxttot, uyttot, uzttot

In [ ]:
segment.mid_lon